# MMF1921 (Summer 2024) - Project 1
 
 The purpose of this program is to implement the following factor models
 
     a) Multi-factor OLS regression
     b) Fama-French 3-factor model
     c) LASSO
     d) Best Subset Selection
 
 and to use these factor models to estimate the asset expected returns and covariance matrix. 
 
These parameters will then be used to test the out-of-sample performance using MVO to construct optimal portfolios.
 
 Use can use this template to write your program.

     Student Name: Kaiwen Shen
     Student ID: 1009970239

In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from functions.BSS import *
from functions.FF import *
from functions.LASSO import *
from functions.MVO import *
from functions.OLS import *
import pandas as pd

adjClose = pd.read_csv("MMF1921_AssetPrices.csv", index_col=0)
factorRet = pd.read_csv("MMF1921_FactorReturns.csv", index_col=0)

In [3]:
adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

In [4]:
#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:, factorRet.columns != 'RF'];

In [5]:
#Identify the tickers and the dates
tickers = adjClose.columns
dates = factorRet.index

In [6]:
# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]

returns = returns - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:, :]

In [7]:
assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

# 2. Define your initial parameters

In [8]:
#Initial budget to invest ($100,000)
initialVal = 100000

#Start of in-sample calibration period
calStart = pd.to_datetime('2008-01-01', format='%Y-%m-%d')
calEnd = calStart + pd.offsets.DateOffset(years=4) - pd.offsets.DateOffset(days=1)

#Start of out-of-sample test period
testStart = pd.to_datetime('2012-01-01', format='%Y-%m-%d')
testEnd = testStart + pd.offsets.DateOffset(years=1) - pd.offsets.DateOffset(days=1)

#Number of investment periods (each investment period is 1 year long)
NoPeriods = 5

#Factor models
#Note: You must populate the functions OLS.py, FF.py, LASSO.py and BSS.py with your own code.
FMList = [OLS, FF, LASSO, BSS]
NoModels = len(FMList)

#Tags for the portfolios under the different factor models
tags = ['OLS portfolio', 'FF portfolio', 'LASSO portfolio', 'BSS portfolio']

# Collecting data for the input of factor combining model

In [9]:
train_period_ret = []
train_period_factor_ret = []
for t in range(NoPeriods):
    # Subset the returns and factor returns corresponding to the current calibration period.
    periodReturns = returns[(calStart <= returns.index) & (returns.index <= calEnd)]
    periodFactRet = factorRet[(calStart <= factorRet.index) & (factorRet.index <= calEnd)]

    # Update your calibration and out-of-sample test periods
    calStart = calStart + pd.offsets.DateOffset(years=1)
    calEnd = calStart + pd.offsets.DateOffset(years=4) - pd.offsets.DateOffset(days=1)
    train_period_ret.append(periodReturns)
    train_period_factor_ret.append(periodFactRet)

In [89]:
mu,q = OLS(train_period_ret[0], train_period_factor_ret[0], 0, 0)

In [90]:
mu

array([ 3.54419148e-02,  1.15344678e-02,  4.94526190e-03,  1.26875542e-02,
        4.94975785e-03, -1.14836519e-03,  5.48639334e-03, -2.73759632e-02,
        7.85426041e-03,  5.94378481e-04,  1.94954270e-02,  1.33638951e-02,
        3.50413316e-03,  1.71467016e-03, -1.14891405e-03, -3.86893190e-05,
        1.03713559e-02, -1.89275404e-03,  3.97532588e-03,  1.05484289e-02])

In [91]:
q

array([[ 0.06253688,  0.01871288,  0.01108002,  0.00235955,  0.00466613,
         0.00315028,  0.00279251,  0.0235531 ,  0.01848801,  0.01405158,
         0.01039243,  0.00512884,  0.00554077,  0.00403706,  0.0023859 ,
         0.0085638 ,  0.00166634,  0.0052649 ,  0.00390217,  0.00285748],
       [ 0.01871288,  0.01821425,  0.00847776,  0.00255756,  0.00372855,
         0.00353981,  0.00293883,  0.01613333,  0.01000229,  0.00852441,
         0.00714534,  0.00413282,  0.00468842,  0.0035787 ,  0.00331545,
         0.008019  ,  0.00187856,  0.0042262 ,  0.00395846,  0.00215132],
       [ 0.01108002,  0.00847776,  0.00711762,  0.00174114,  0.00217173,
         0.00225293,  0.00156833,  0.01203873,  0.00662717,  0.00560314,
         0.00500908,  0.00275987,  0.00298328,  0.00238113,  0.00189426,
         0.00552871,  0.00129356,  0.00243228,  0.00226734,  0.00128182],
       [ 0.00235955,  0.00255756,  0.00174114,  0.00200683,  0.00092574,
         0.00093609,  0.00089915,  0.00405077,  

In [47]:
coef = np.linalg.inv(train_period_factor_ret[0].T @ train_period_factor_ret[0]) @ train_period_factor_ret[0].T @ \
train_period_ret[0]

In [62]:
np.dot(train_period_factor_ret[0],coef)

array([[ 6.13032527e-02, -1.35192662e-02, -6.60132756e-02,
        -2.30356825e-02, -2.91346486e-02, -3.43280494e-02,
         2.29457187e-02, -6.11695135e-02,  1.18266933e-01,
         8.62564231e-02, -2.05811999e-01, -8.62512981e-03,
         2.64761450e-02, -3.54958009e-02, -6.95770339e-02,
        -1.45026546e-01, -1.64856499e-02, -7.53035902e-02,
        -6.86107188e-02, -4.77836885e-02],
       [-2.13155988e-01, -5.22409154e-02, -3.44485476e-02,
         3.54649655e-03, -2.39183007e-02, -2.53269131e-03,
         1.96018956e-03, -8.97554218e-02, -8.06475164e-02,
        -5.43461000e-02, -8.23792999e-02, -9.06716505e-04,
        -1.02000886e-02, -5.94355895e-03, -2.09175549e-02,
        -1.94076920e-02, -7.88890446e-03, -4.53096339e-02,
        -3.39567498e-02, -9.56684050e-03],
       [-9.98815356e-03,  6.67907946e-04, -1.48229192e-02,
         6.75233790e-03,  1.90095888e-03,  4.70164086e-03,
         2.56630112e-02, -5.60732243e-02,  1.64048807e-03,
         1.30443807e-02, -2.0

In [72]:
from functions.helper import r_squared, adjusted_r_squared

In [70]:
r_squared(train_period_ret[0], np.dot(train_period_factor_ret[0], coef))

F       0.614742
CAT     0.785947
DIS     0.784618
MCD     0.393886
KO      0.446219
PEP     0.432556
WMT     0.562854
C       0.789891
WFC     0.776620
JPM     0.795554
AAPL    0.565769
IBM     0.480355
PFE     0.535740
JNJ     0.503251
XOM     0.646015
MRO     0.605605
ED      0.141824
T       0.510315
VZ      0.534462
NEM     0.166046
dtype: float64

In [74]:
adjusted_r_squared(train_period_ret[0], np.dot(train_period_factor_ret[0], coef), train_period_factor_ret[0])

F       0.535715
CAT     0.742039
DIS     0.740437
MCD     0.269555
KO      0.332623
PEP     0.316158
WMT     0.473183
C       0.746792
WFC     0.730798
JPM     0.753616
AAPL    0.476696
IBM     0.373761
PFE     0.440507
JNJ     0.401354
XOM     0.573403
MRO     0.524704
ED     -0.034212
T       0.409867
VZ      0.438967
NEM    -0.005022
dtype: float64

In [78]:
np.diag(np.var(train_period_ret[0]- np.dot(train_period_factor_ret[0], coef),axis=0))

array([[0.02372092, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.00381403, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.00148048, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00109953, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

In [81]:
np.cov(train_period_factor_ret[0].values)

array([[ 2.51860500e-03, -2.97721429e-05, -3.22780714e-04, ...,
         2.41979286e-04, -3.21134643e-04, -2.16412500e-04],
       [-2.97721429e-05,  1.10389643e-03,  2.33096429e-04, ...,
        -6.28164286e-05,  5.22658214e-04, -1.25999643e-04],
       [-3.22780714e-04,  2.33096429e-04,  2.90530714e-04, ...,
        -5.13617857e-04,  6.26896429e-05,  9.78260714e-05],
       ...,
       [ 2.41979286e-04, -6.28164286e-05, -5.13617857e-04, ...,
         3.22207357e-03, -3.51092500e-04, -5.23968929e-04],
       [-3.21134643e-04,  5.22658214e-04,  6.26896429e-05, ...,
        -3.51092500e-04,  4.73356964e-04,  2.28946429e-06],
       [-2.16412500e-04, -1.25999643e-04,  9.78260714e-05, ...,
        -5.23968929e-04,  2.28946429e-06,  1.75578393e-04]])

In [84]:
F = np.cov(train_period_factor_ret[0].values, rowvar=False)

In [85]:
coef.T @ F @ coef 

,F,CAT,DIS,MCD,KO,PEP,WMT,C,WFC,JPM,AAPL,IBM,PFE,JNJ,XOM,MRO,ED,T,VZ,NEM
F,0.036748,0.017900,0.011383,0.003070,0.004681,0.003310,0.002806,0.025025,0.018305,0.014080,0.011404,0.005794,0.005959,0.004294,0.002482,0.009291,0.002307,0.004887,0.003939,0.003254
CAT,0.017900,0.014134,0.008599,0.002839,0.003741,0.003600,0.002952,0.016659,0.009944,0.008537,0.007547,0.004397,0.004851,0.003678,0.003352,0.008294,0.002131,0.004084,0.003978,0.002312
DIS,0.011383,0.008599,0.005660,0.001746,0.002196,0.002244,0.001597,0.011839,0.006677,0.005609,0.005018,0.002767,0.002972,0.002375,0.001891,0.005485,0.001291,0.002458,0.002287,0.001297
MCD,0.003070,0.002839,0.001746,0.000973,0.000975,0.000912,0.000958,0.003584,0.001774,0.001542,0.001628,0.000882,0.001119,0.000904,0.000991,0.001759,0.000511,0.000844,0.000893,0.000950
KO,0.004681,0.003741,0.002196,0.000975,0.001319,0.001064,0.001006,0.004307,0.002163,0.001949,0.002386,0.001149,0.001414,0.001063,0.001390,0.002341,0.000672,0.001342,0.001393,0.001244
PEP,0.003310,0.003600,0.002244,0.000912,0.001064,0.001155,0.001018,0.004807,0.002271,0.002238,0.001938,0.001118,0.001445,0.001080,0.001138,0.002338,0.000606,0.001126,0.001152,0.000644
WMT,0.002806,0.002952,0.001597,0.000958,0.001006,0.001018,0.001341,0.004229,0.002418,0.002394,0.001050,0.000780,0.001492,0.000878,0.001011,0.001325,0.000503,0.000831,0.000878,0.000569
C,0.025025,0.016659,0.011839,0.003584,0.004307,0.004807,0.004229,0.036880,0.021114,0.017884,0.007420,0.005032,0.008331,0.004966,0.003183,0.009459,0.002519,0.004051,0.003349,0.001218
WFC,0.018305,0.009944,0.006677,0.001774,0.002163,0.002271,0.002418,0.021114,0.014876,0.011627,0.003249,0.002491,0.004614,0.002446,0.000884,0.003767,0.001261,0.001975,0.001323,-0.000202
JPM,0.014080,0.008537,0.005609,0.001542,0.001949,0.002238,0.002394,0.017884,0.011627,0.010191,0.003072,0.002532,0.004330,0.002276,0.000957,0.003811,0.001069,0.001793,0.001304,-0.000246


In [60]:
coef.shape

(8, 20)

# 3. Construct and rebalance your portfolios

Here you will estimate your input parameters (exp. returns and cov. matrix etc) from the Fama-French factor models.
You will have to re-estimate your parameters at the start of each rebalance period, and then re-optimize and rebalance your portfolios accordingly.

Ensure you re-initialize the dates above if you run this cell repeatedly. 

In [ ]:
# Initiate counter for the number of observations per investment period
toDay = 0

# Preallocate the space for the per period value of the portfolios 
currentVal = {i: np.zeros(NoPeriods) for i in range(NoModels)}

# Number of assets
n = len(tickers)

# Preallocate space for the portfolio weights
x = {i: np.zeros([n, NoPeriods]) for i in range(NoModels)}

# Initialize dictionaries to hold Q, mu and the number of shares 
# for each model. These are overwritten at each rebalancing point
mu = {}
Q = {}
NoShares = {}

# Empty lists to measure the value of the portfolio over the period
portfValue = {i: [] for i in range(NoModels)}

#--------------------------------------------------------------------------
# Set the value of lambda and K for the LASSO and BSS models, respectively
#--------------------------------------------------------------------------
lambda_ = 0.5
K = 4

for t in range(NoPeriods):
    # Subset the returns and factor returns corresponding to the current calibration period.
    periodReturns = returns[(calStart <= returns.index) & (returns.index <= calEnd)]
    periodFactRet = factorRet[(calStart <= factorRet.index) & (factorRet.index <= calEnd)]

    current_price_idx = (calEnd - pd.offsets.DateOffset(days=7) <= adjClose.index) & (adjClose.index <= calEnd)
    currentPrices = adjClose[current_price_idx]

    # Subset the prices corresponding to the current out-of-sample test period.
    periodPrices_idx = (testStart <= adjClose.index) & (adjClose.index <= testEnd)
    periodPrices = adjClose[periodPrices_idx]

    assert len(currentPrices) == 1
    # Set the initial value of the portfolio or update the portfolio value
    if t == 0:
        for i in range(NoModels):
            currentVal[i][0] = initialVal  # all models start with the same amount of $
    else:
        for i in range(NoModels):
            currentVal[i][t] = (currentPrices @ NoShares[i].values.T).squeeze()

    # Update counter for the number of observations per investment period
    fromDay = toDay
    toDay = toDay + len(periodPrices)

    # Calculate 'mu' and 'Q' using the 4 factor models.
    # Note: You need to write the code for the 4 factor model functions. 
    for i in range(NoModels):
        mu[i], Q[i] = FMList[i](periodReturns, periodFactRet, lambda_, K)

    # Optimize your portfolios to get the weights 'x'
    # Note: You need to write the code for MVO with no short sales
    for i in range(NoModels):
        # Define the target return as the geometric mean of the market 
        # factor for the current calibration period
        targetRet = gmean(periodFactRet.iloc[:, 0] + 1) - 1

        x[i][:, t] = MVO(mu[i], Q[i], targetRet)

        # Calculate the optimal number of shares of each stock you should hold
    for i in range(NoModels):
        # Number of shares your portfolio holds per stock
        NoShares[i] = x[i][:, t] * currentVal[i][t] / currentPrices

        # Weekly portfolio value during the out-of-sample window
        portfValue[i].append(periodPrices @ NoShares[i].values.T)

    # Update your calibration and out-of-sample test periods
    calStart = calStart + pd.offsets.DateOffset(years=1)
    calEnd = calStart + pd.offsets.DateOffset(years=4) - pd.offsets.DateOffset(days=1)

    testStart = testStart + pd.offsets.DateOffset(years=1)
    testEnd = testStart + pd.offsets.DateOffset(years=1) - pd.offsets.DateOffset(days=1)

for i in range(NoModels):
    portfValue[i] = pd.concat(portfValue[i], axis=0)

# Overwrite into a dataframe
portfValue = pd.DataFrame([portfValue[i].values.squeeze() for i in range(NoModels)],
                          index=tags, columns=portfValue[0].index).T


# 4. Results

In [ ]:
#--------------------------------------------------------------------------
# 4.1 Evaluate any measures of fit of the regression models to assess their
# in-sample quality. You may want to modify Section 3 of this program to
# calculate the quality of fit each time the models are recalibrated.
#--------------------------------------------------------------------------

#--------------------------------------------------------------------------
# 4.2 Calculate the portfolio average return, variance (or standard 
# deviation), and any other performance and/or risk metric you wish to 
# include in your report.
#--------------------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------------------
# 4.3 Plot the portfolio wealth evolution 
# 
# Note: The code below plots all portfolios onto a single plot. However,
# you may want to split this into multiple plots for clarity, or to
# compare a subset of the portfolios. 
#--------------------------------------------------------------------------
# Calculate the dates of the out-of-sample period

fig = plt.figure(1)
portfValue.plot(title='Portfolio wealth evolution',
                ylabel='Total wealth',
                figsize=(6, 3),
                legend=True)
plt.savefig("images/wealth.svg")

#--------------------------------------------------------------------------
# 4.4 Plot the portfolio weights period-over-period
#--------------------------------------------------------------------------
# OLS Portfolio weights

fig2 = plt.figure(2)
x[0][x[0] < 0] = 0
weights = pd.DataFrame(x[0][(x[0] > 0).any(axis=1)], index=tickers[(x[0] > 0).any(axis=1)])
weights.columns = [col + 1 for col in weights.columns]
weights.T.plot.area(title='Portfolio weights',
                    ylabel='Weights', xlabel='Rebalance Period',
                    figsize=(6, 3),
                    legend=True, stacked=True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.savefig("images/weights.svg")
#
# ###########################################################################
# # Program End